In [2]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

In [3]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [4]:
def create_bucket(bucket):
    import logging

    try:
        s3.create_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError as e:
        logging.error(e)
        return 'Bucket ' + bucket + ' could not be created.'
    return 'Created or already exists ' + bucket + ' bucket.'


In [6]:
create_bucket('qtm-350-project-taxis-pls-show-in-my-s3')

'Created or already exists qtm-350-project-taxis-pls-show-in-my-s3 bucket.'

In [22]:
import pandas as pd
data = pd.read_csv("yellow_tripdata_2019-01.csv")
data_green = pd.read_csv("green_tripdata_2019-01.csv")

In [23]:
data = data.sample(frac = 0.1)

In [54]:
data_green.head()


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,time,class,tip_share
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,...,0.0,NaN,0.3,4.30,2,1,NaN,00:01:28,0,0.000000
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,...,0.0,NaN,0.3,7.30,2,1,NaN,00:06:16,0,0.000000
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,...,0.0,NaN,0.3,5.80,1,1,NaN,00:04:27,0,0.000000
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,...,0.0,NaN,0.3,19.71,1,1,NaN,00:18:34,0,0.219259
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,...,0.0,NaN,0.3,19.30,2,1,NaN,00:20:37,0,0.000000


In [35]:
data['class'] = 1

In [36]:
data_green['class'] = 0

In [6]:
import re

data_green['lpep_pickup_datetime'] = data_green['lpep_pickup_datetime'].str.extract('(\d{2}:\d{2}:\d{2})')
data_green['lpep_dropoff_datetime'] = data_green['lpep_dropoff_datetime'].str.extract('(\d{2}:\d{2}:\d{2})')
#data_green['time'] = data_green['lpep_dropoff_datetime'] - data_green['lpep_pickup_datetime']

In [7]:
data['tpep_pickup_datetime'] = data['tpep_pickup_datetime'].str.extract('(\d{2}:\d{2}:\d{2})')
data['tpep_dropoff_datetime'] = data['tpep_dropoff_datetime'].str.extract('(\d{2}:\d{2}:\d{2})')

In [25]:
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [26]:
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
data_green['lpep_pickup_datetime'] = pd.to_datetime(data_green['lpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
data_green['lpep_dropoff_datetime'] = pd.to_datetime(data_green['lpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')

In [27]:
(data_green['lpep_pickup_datetime'])
data_green['lpep_dropoff_datetime']

0        2018-12-21 15:18:57
1        2019-01-01 00:16:32
2        2019-01-01 00:31:38
3        2019-01-01 01:04:54
4        2019-01-01 00:39:43
5        2019-01-01 00:19:09
6        2019-01-01 01:00:01
7        2019-01-01 00:30:50
8        2019-01-01 00:39:46
9        2019-01-01 01:19:02
10       2019-01-01 00:56:42
11       2019-01-01 00:21:00
12       2019-01-01 00:45:50
13       2019-01-01 01:20:10
14       2019-01-01 00:17:50
15       2019-01-01 00:25:29
16       2019-01-01 00:52:59
17       2019-01-01 00:54:19
18       2019-01-01 00:00:57
19       2019-01-01 00:50:23
20       2019-01-01 01:04:10
21       2019-01-01 00:41:18
22       2019-01-01 01:01:33
23       2019-01-01 00:42:32
24       2019-01-01 01:18:53
25       2019-01-01 00:40:00
26       2019-01-01 00:55:49
27       2019-01-01 00:08:45
28       2019-01-01 00:43:17
29       2019-01-01 00:53:03
                 ...        
630888   2019-02-01 00:14:46
630889   2019-01-31 23:26:20
630890   2019-01-31 23:53:56
630891   2019-

In [28]:
data['time'] = data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']
data_green['time'] = data_green['lpep_dropoff_datetime'] - data_green['lpep_pickup_datetime']

In [30]:
data_green['time']

0        00:01:28
1        00:06:16
2        00:04:27
3        00:18:34
4        00:20:37
5        00:06:34
6        00:12:06
7        00:18:03
8        00:23:23
9        00:21:00
10       00:19:42
11       00:07:12
12       00:25:51
13       00:22:13
14       00:08:48
15       00:03:17
16       00:21:04
17       00:23:59
18       00:02:51
19       00:10:06
20       00:12:54
21       00:10:50
22       00:34:34
23       00:03:23
24       00:43:40
25       00:04:08
26       00:07:22
27       00:07:24
28       00:18:38
29       00:08:38
           ...   
630888   00:16:05
630889   00:07:48
630890   00:09:40
630891   00:04:37
630892   00:07:04
630893   00:08:03
630894   00:15:01
630895   00:06:39
630896   00:07:11
630897   00:14:51
630898   00:26:42
630899   00:16:06
630900   00:02:56
630901   00:09:12
630902   00:06:27
630903   00:05:49
630904   00:17:54
630905   00:02:35
630906   00:09:23
630907   00:06:35
630908   00:08:17
630909   00:04:01
630910   00:08:31
630911   00:06:27
630912   0

In [89]:
data['tip_share'] = data['tip_amount']/data['fare_amount']
data_green['tip_share'] = data_green['tip_amount']/data_green['fare_amount']
data_green_selected = data_green[['time', 'trip_distance', 'passenger_count', 'fare_amount', 'class', 'tip_share']]
data_selected = data[['time', 'trip_distance', 'passenger_count', 'fare_amount', 'class', 'tip_share']]

In [98]:
data_green_selected

,time,trip_distance,passenger_count,fare_amount,class,tip_share
0,00:01:28,0.00,5,3.0,0,0.000000
1,00:06:16,0.86,2,6.0,0,0.000000
2,00:04:27,0.66,2,4.5,0,0.000000
3,00:18:34,2.68,2,13.5,0,0.219259
4,00:20:37,4.53,1,18.0,0,0.000000
5,00:06:34,1.05,1,6.5,0,0.000000
6,00:12:06,3.77,1,13.5,0,0.000000
7,00:18:03,4.10,1,16.0,0,0.000000
8,00:23:23,7.75,1,25.5,0,0.000000
9,00:21:00,3.68,1,15.5,0,0.000000


In [90]:
data_total = pd.concat([data_green_selected, data_selected])

In [61]:
data_total.to_csv('taxi_data.csv', encoding = 'utf-8')

# Actual ML stuff

In [57]:
import boto3
import io
import matplotlib.pyplot as plt
import numpy as np 
import os
import pandas as pd 

import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_deserializer

In [65]:
# Set data locations
bucket = 'smworkshop-alex-fan'                        # replace this with your own bucket
prefix = 'sagemaker/qtm-350-project'   
s3_train_key = '{}/train/recordio-pb-data'.format(prefix)
s3_train_path = os.path.join('s3://', bucket, s3_train_key)
local_raw_data = 'taxi_data.csv'
role = get_execution_role()

In [105]:
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    print(obj.key)

sagemaker/linear-loss-SGD/class_weights/output/linear-learner-2019-11-08-17-42-23-642/output/model.tar.gz
sagemaker/linear-loss-SGD/class_weights/output/linear-learner-2019-11-13-17-47-25-048/output/model.tar.gz
sagemaker/linear-loss-SGD/class_weights/output/linear-learner-2019-11-13-18-21-15-385/output/model.tar.gz
sagemaker/linear-loss-SGD/defaults/output/linear-learner-2019-11-08-17-27-04-759/output/model.tar.gz
sagemaker/linear-loss-SGD/defaults/output/linear-learner-2019-11-13-17-29-09-598/output/model.tar.gz
sagemaker/linear-loss-SGD/defaults/output/linear-learner-2019-11-13-18-02-40-843/output/model.tar.gz
sagemaker/linear-loss-SGD/train/recordio-pb-data
sagemaker/qtm-350-project/train/recordio-pb-data
sagemaker/videogames_xgboost/train/train.libsvm
sagemaker/videogames_xgboost/validation/validation.libsvm
sagemaker/videogames_xgboost/videogames-xgboost-2019-11-01-16-44-34/output/model.tar.gz


In [96]:
god = []
for i in data_total['time']:
    god.append(i.total_seconds())
    
god


AttributeError: 'DataFrame' object has no attribute 'total'

In [101]:
data_total['time_real(s)'] = god
data_total = data_total[['trip_distance', 'passenger_count', 'time_real(s)', 'fare_amount',
                        'class', 'tip_share']]
data_total

,trip_distance,passenger_count,time_real(s),fare_amount,class,tip_share
0,0.00,5,88.0,3.0,0,0.000000
1,0.86,2,376.0,6.0,0,0.000000
2,0.66,2,267.0,4.5,0,0.000000
3,2.68,2,1114.0,13.5,0,0.219259
4,4.53,1,1237.0,18.0,0,0.000000
5,1.05,1,394.0,6.5,0,0.000000
6,3.77,1,726.0,13.5,0,0.000000
7,4.10,1,1083.0,16.0,0,0.000000
8,7.75,1,1403.0,25.5,0,0.000000
9,3.68,1,1260.0,15.5,0,0.000000


In [104]:
raw_data = data_total.as_matrix()
np.random.seed(0)
np.random.shuffle(raw_data)
train_size = int(raw_data.shape[0] * 0.7)
train_features  = raw_data[:train_size, :-1]
train_labels = raw_data[:train_size, -1]
test_features = raw_data[train_size:, :-1]
test_labels = raw_data[train_size:, -1]

# Convert the processed training data to protobuf and write to S3 for linear learner
vectors = np.array([t.tolist() for t in train_features]).astype('float32')
labels = np.array([t.tolist() for t in train_labels]).astype('float32')
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(s3_train_key).upload_fileobj(buf)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [106]:
from sagemaker.amazon.amazon_estimator import get_image_uri

def predictor_from_hyperparams(s3_train_data, hyperparams, output_path):
    """
    Create an Estimator from the given hyperparams, fit to training data, and return a deployed predictor
    """
    # specify algorithm containers and instantiate an Estimator with given hyperparams
    container = get_image_uri(boto3.Session().region_name, 'linear-learner')

    linear = sagemaker.estimator.Estimator(container,
        role,
        train_instance_count=1,
        train_instance_type='ml.m4.xlarge',
        output_path=output_path,
        sagemaker_session=sagemaker.Session())
    linear.set_hyperparameters(**hyperparams)
    # train model
    linear.fit({'train': s3_train_data})
    # deploy a predictor
    linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
    linear_predictor.content_type = 'text/csv'
    linear_predictor.serializer = csv_serializer
    linear_predictor.deserializer = json_deserializer
    return linear_predictor

In [107]:
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [ ]:
# Training a binary classifier with default settings: logistic regression
defaults_hyperparams = {
    'feature_dim': 5,
    'predictor_type': 'regressor',
    'epochs': 10
}
defaults_output_path = 's3://{}/{}/defaults/output'.format(bucket, prefix)
defaults_predictor = predictor_from_hyperparams(s3_train_path, defaults_hyperparams, defaults_output_path)

2019-12-04 05:30:32 Starting - Starting the training job...
2019-12-04 05:30:35 Starting - Launching requested ML instances...